# Retrieve underlying RICs of chain RIC

## Method 2) Get chain constituents with Refinitiv Data Platform API
Here comes an easier and simpler way! with only one endpoint in RDP API, we can retrieve constituent lists of Chain RIC in one call (not counting a call to retrieve access token though). The example of this API call for this endpoint can be found in the API Playground: /data/pricing/chains/v1/

With this method, we can retrieve the chain constituents list with just one API call instead of looping through each Chain Record and sending requests to the server multiple times. We also don't need to use WebSocket to make a connection to the Real-time platform so no need to implement the message handler functions. Lastly, we don't have to know the chain structure and handle each chain's record template because this endpoint is already handling it for us!

In [1]:
import rdpToken
import http.client
import json
import time

base_ric = '0#.FTSE'

def get_underlying_rics_rdp():
    try:
        auth_token = rdpToken.getToken()
        
        print(f'Retrieving underlying RICs of chain RIC: ', base_ric)
        conn = http.client.HTTPSConnection("api.refinitiv.com")
        payload = ''
        headers = {
            'Authorization': 'Bearer ' + auth_token
        }
        conn.request('GET', f'/data/pricing/chains/v1/?universe={base_ric}', payload, headers)
        r = conn.getresponse()
        data = r.read()

        data = json.loads(data.decode("utf-8"))

    except requests.exceptions.RequestException as e:
        print('Refinitiv Data Platform authentication exception failure:', e)

    if r.status == 200:
        print('Underlying RICs of chain are successfully retrieved')

    elif r.status in [400, 401, 403, 404, 410, 451, 503]:
        print('Error retrieving underlying RICs (to be retried):', r.status, r.reason)
        time.sleep(5)
        return get_underlying_rics_rdp()

    elif data == b'':
        print('Error retrieving underlying RICs (to be retried): get empty response')
        time.sleep(5)
        return get_underlying_rics_rdp()

    else:
        # Retry the request to the API gateway
        print('Retrieval of underlying RICs of chain HTTP code:', r.status, r.reason)
        time.sleep(5)
        print('Retry change request')
        return get_underlying_rics_rdp()
    return data

In [2]:
if __name__ == "__main__":
    data = get_underlying_rics_rdp()
    print(str(len(data['data']['constituents'])) + ' underlying RICs are retrieved, which are')
    print(data['data']['constituents'])

Existing token read from: token.txt
Retrieving underlying RICs of chain RIC:  0#.FTSE
Underlying RICs of chain are successfully retrieved
102 underlying RICs are retrieved, which are
['.FTSE', '.AD.FTSE', 'AAF.L', 'AAL.L', 'ABDN.L', 'ABF.L', 'ADML.L', 'AHT.L', 'ANTO.L', 'AUTOA.L', 'AV.L', 'AVST.L', 'AVV.L', 'AZN.L', 'BAES.L', 'BARC.L', 'BATS.L', 'BDEV.L', 'BKGH.L', 'BLND.L', 'BMEB.L', 'BNZL.L', 'BP.L', 'BRBY.L', 'BT.L', 'CCH.L', 'CNA.L', 'CPG.L', 'CRDA.L', 'CRH.L', 'DCC.L', 'DGE.L', 'DPH.L', 'EDV.L', 'ENT.L', 'EXPN.L', 'FLTRF.L', 'FRES.L', 'GLEN.L', 'GSK.L', 'HIK.L', 'HLMA.L', 'HLN.L', 'HRGV.L', 'HSBA.L', 'HWDN.L', 'ICAG.L', 'ICP.L', 'IHG.L', 'III.L', 'IMB.L', 'INF.L', 'ITRK.L', 'JD.L', 'KGF.L', 'LAND.L', 'LGEN.L', 'LLOY.L', 'LSEG.L', 'MGGT.L', 'MNDI.L', 'MNG.L', 'MRON.L', 'NG.L', 'NWG.L', 'NXT.L', 'OCDO.L', 'PHNX.L', 'PRU.L', 'PSHP.L', 'PSN.L', 'PSON.L', 'REL.L', 'RIO.L', 'RKT.L', 'RMV.L', 'RR.L', 'RS1R.L', 'RTO.L', 'SBRY.L', 'SDR.L', 'SGE.L', 'SGRO.L', 'SHEL.L', 'SJP.L', 'SKG.L', 'SM

## Thank you